In [ ]:
import tensorflow as tf
import time,numpy
from sys import getsizeof

In [ ]:
A = numpy.random.random([4001,4001])
#print(A)
mem = getsizeof(A)
print("Memory allocated: {:.3f} Megabytes".format(mem/1024/1024))

## Svd

### Numpy, cpu

In [ ]:
start = time.time()
u,s,v = numpy.linalg.svd(A)
print("Temps d'exécution: {:.3f} secondes".format(time.time()-start))
print(s[:5])

### Tensorflow, cpu

In [ ]:
sess = tf.InteractiveSession()
sess.as_default()

with tf.device('/cpu:0'):
    s, v, u = tf.svd(A)

start = time.time()
for i in range(1):
    res = s.eval()
    #res = sess.run(s)
print("Temps d'exécution: {:.3f} secondes".format(time.time()-start))
print(res[:5])

### Tensorflow, gpu

In [ ]:
sess = tf.InteractiveSession()
sess.as_default()

with tf.device('/gpu:0'):
    s, v, u = tf.svd(A)

start = time.time()
for i in range(1):
    res = s.eval()
    #res = sess.run(s)
print("Temps d'exécution: {:.3f} secondes".format(time.time()-start))
print(res[:5])

## Multiplication de matrices

### Numpy, cpu

In [ ]:
start = time.time()
for i in range(1):
    B = numpy.dot(A,A)
print("Temps d'exécution: {:.3f} secondes".format(time.time()-start))
print(B[:3,:3])

### Tensorflow, cpu

In [ ]:
sess = tf.InteractiveSession()
sess.as_default()

with tf.device('/cpu:0'):
    B = tf.matmul(A,A)

start = time.time()
for i in range(1):
    res = sess.run(B)
print("Temps d'exécution: {:.3f} secondes".format(time.time()-start))

devices = sess.list_devices()
for d in devices:
    print(d.name)
sess.close()

### Tensorflow, gpu

In [ ]:
sess = tf.InteractiveSession()
sess.as_default()

with tf.device('/gpu:0'):
    B = tf.matmul(A,A)

start = time.time()
for i in range(1):
    res = sess.run(B)
print("Temps d'exécution: {:.3f} secondes".format(time.time()-start))

devices = sess.list_devices()
for d in devices:
    print(d.name)
sess.close()

In [ ]:
sess = tf.InteractiveSession()
sess.as_default()

with tf.device('/cpu:0'):
    B_cpu = tf.matmul(A,A)
with tf.device('/gpu:0'):
    B_gpu = tf.matmul(A,A)
with tf.device('/gpu:0'):
    B_gpu2 = tf.matmul(A,A)
    
start = time.time()
for i in range(1):
    res_gpu2 = sess.run(B_gpu2)
print("Temps d'exécution (gpu): {:.3f} secondes".format(time.time()-start))

start = time.time()
for i in range(1):
    res_cpu = sess.run(B_cpu)
print("Temps d'exécution (cpu): {:.3f} secondes".format(time.time()-start))

start = time.time()
for i in range(1):
    res_gpu = sess.run(B_gpu)
print("Temps d'exécution (gpu): {:.3f} secondes".format(time.time()-start))

devices = sess.list_devices()
for d in devices:
    print(d.name)
sess.close()

In [ ]:
import numpy as np
import tensorflow as tf
import time

n = 4001
X = tf.constant(np.array(np.random.randn(n,n), dtype = np.float32), dtype = tf.float32)
Y = tf.matmul(X, tf.transpose(X))

init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)

a = time.time()
sess.run(Y)
print (time.time() - a)
sess.close()

In [ ]:
import numpy as np
import tensorflow as tf
import time

size = 8001
x = np.array(np.random.randn(size, size), dtype = np.float32)

X = tf.placeholder(tf.float32, shape=(size, size), name=None)
Y = tf.matmul(X, X)

sess = tf.Session()

a = time.time()
sess.run(Y, feed_dict={X: x})
print (time.time() - a)

In [ ]:
import numpy as np
import time
np.random.seed(1)
n = 8001
x = np.array(np.random.randn(n,n), dtype = np.float32)
a = time.time(); x.dot(x); print (time.time() - a)

In [ ]:
import operateurs_t

def U(y,t):
    return 1-y**2

Re = 10000
alpha = 0.6
N = 7
t = 30

# tensorflow

[a,b] = operateurs_t.buildAB_forward(Re, alpha, N, U, t)
A = tf.placeholder(tf.complex128, shape=(N-4, N-4))
B = tf.placeholder(tf.complex128, shape=(N-4, N-4))

with tf.device('/gpu:0'):
    B_inv = tf.matrix_inverse(B)
    M = tf.multiply(-1j*alpha,tf.matmul(B_inv,A))

sess = tf.Session()
numpyM = sess.run(M, feed_dict={A: a, B: b})

print(numpyM)

# numpy

[A,B] = operateurs_t.buildAB_forward(Re, alpha, N, U, t)
B_inv = numpy.linalg.inv(B)
M = -1j*alpha*numpy.dot(B_inv,A)

print(M)

In [ ]:
import operateurs_t

def U(y,t):
    return 1-y**2

Re = 10000
alpha = 0.6
N = 7
t = 30

# tensorflow
def f(t,L,Re,alpha,N,U):
    
    [A,B] = operateurs_t.buildAB_forward(Re, alpha, N, U, t)
    B = B*(1+0.0j)

    with tf.device('/gpu:0'):
        B_inv = tf.matrix_inverse(B)
        M = tf.multiply(-1j*alpha,tf.matmul(B_inv,A))
        L_new = tf.matmul(L,M)
    
    sess = tf.Session()
    L_new_n = sess.run(L_new)
    
    return L_new_n

L = numpy.eye(N-4)*(1+0.0j)
print(f(t,L,Re,alpha,N,U))

# numpy

[A,B] = operateurs_t.buildAB_forward(Re, alpha, N, U, t)
B_inv = numpy.linalg.inv(B)
M = -1j*alpha*numpy.dot(B_inv,A)

print(M)

In [ ]:
a = 2*numpy.eye(2)
b = numpy.eye(2)
b[1,0] = 1
cT = tf.matmul(a,b)
sess = tf.Session()
c = sess.run(cT)
print(c)

In [ ]:
import operateurs_t,scipy,numpy
import tensorflow as tf

def U(y,t):
    return 1-y**2

Re = 10000
alpha = 0.6
N = 41
t_max = 0.1
dt = 0.1
step = 20

st,t_vec = svd_tf_RK4(Re,alpha,N,t_max,dt,step,U)
print(st)

In [ ]:
def f(t,L,Re,alpha,N,U):
    
    [A,B] = operateurs_t.buildAB_forward(Re, alpha, N, U, t)
    B = B*(1+0.0j)

    with tf.device('/gpu:0'):
        B_inv = tf.matrix_inverse(B)
        M = tf.multiply(-1j*alpha,tf.matmul(B_inv,A))
        L_new = tf.matmul(L,M)
    
    sess = tf.Session()
    sess.run(L_new)
    
    return L_new

def svd_tf_RK4(Re,alpha,N,t_max,dt,step,U):

    nt = int(t_max/dt)+1
    st = numpy.empty((int(nt/step)+1,N-4)) # valeurs singulières
    t_vec = numpy.empty((int(nt/step)+1,1))

    t = 0
    L = scipy.eye(N-4)*(1+0.0j)
    t_vec[0] = 0
    st[0,:] = 1
    
    sess = tf.Session()

    for i in range(1,nt+1):
    
        # La matrice L=exp(Mt) est avancé de dt dans le temps avec RK4
        with tf.device('/gpu:0'):
            k1 = f(t,L,Re,alpha,N,U)
            k2 = f(t+dt/2,tf.add(L,tf.multiply(k1,1/2*(1+0.0j))),Re,alpha,N,U)
            k3 = f(t+dt/2,tf.add(L,tf.multiply(k2,1/2*(1+0.0j))),Re,alpha,N,U)
            k4 = f(t+dt,tf.add(L,k3),Re,alpha,N,U)
    
            # L = L + dt/6*(k1+2*k2+2*k3+k4)
            temp = tf.add(k1,tf.add(tf.multiply(2+0.0j,k2),tf.add(tf.multiply(2+0.0j,k3),k4)))
            L = tf.add(L,tf.multiply(dt/6*(1+0.0j),temp))
    
        t =t+dt
        sess.run(L)
        
        # Calcul de svd
        if i%step==0:
            with tf.device('/cpu:0'):
                s = tf.svd(L,compute_uv=False)
            
            index = int(i/step)
            s_numpy = sess.run(s)
            st[index,:] = s_numpy
            t_vec[index] = t
            
    return st,t_vec

### Step by step w/ tensorflow

In [ ]:
import operateurs_t,scipy,numpy
import tensorflow as tf

def U(y,t):
    return 1-y**2

In [ ]:
Re = 10000
alpha = 0.6
N = 401
t_max = 0.1
dt = 0.1
step = 20

t = 0.1
L = scipy.eye(N-4)*(1+0.0j)

[a,b] = operateurs_t.buildAB_forward(Re, alpha, N, U, t)
b = b*(1+0.0j)
A = tf.placeholder(tf.complex128, shape=(N-4, N-4))
B = tf.placeholder(tf.complex128, shape=(N-4, N-4))

with tf.device('/gpu:0'):
    B_inv = tf.matrix_inverse(B)
    M = tf.multiply(-1j*alpha,tf.matmul(B_inv,A))
    L_new = tf.matmul(L,M)
    
sess = tf.Session()
res = sess.run(B_inv,feed_dict={A:a, B:b})
print(res)

In [ ]:
import operateurs_t,scipy,numpy
import tensorflow as tf

def U(y,t):
    return 1-y**2

Re = 10000
alpha = 0.6
N = 401
t = 0.1

tf.InteractiveSession()

[a,b] = operateurs_t.buildAB_forward(Re, alpha, N, U, t)
b = b *(1+0.0j)

B = tf.placeholder(tf.complex128, shape=(N-4, N-4))
#with tf.device('/gpu:0'):
B_inv = tf.matrix_inverse(B).eval(feed_dict={B: b})

In [ ]:
import numpy
import tensorflow as tf

sess = tf.Session()

N = 401

a = numpy.ones((N,N))
A = tf.placeholder(tf.float64, shape=(N, N))
B = A

for i in range(1000):
    B = tf.add(B,A)
    
sess.run(B, feed_dict={A:a})

### Tensorflow learning examples

In [ ]:
import numpy,scipy
import tensorflow as tf
import operateurs_t

In [ ]:
def f(t,L,Re,alpha,N,U):
        
    [a,b] = operateurs_t.buildAB_forward(Re, alpha, N, U, t)
    b = b*(1+0.0j)
    
    A = tf.constant(a)
    B = tf.constant(b)

    with tf.device('/gpu:0'):
        B_inv = tf.matrix_inverse(B)
        M = tf.multiply(-1j*alpha,tf.matmul(B_inv,A))
        L_new = tf.matmul(L,M)
    
    return L_new

In [ ]:
def U(y,t):
    return 1-y**2

Re = 10000
alpha = 0.6
N = 20

t = 0.1
L = tf.constant(scipy.eye(N-4)*(1+0.0j))

##############################
sess = tf.Session()

res = f(t,L,Re,alpha,N,U)
print(sess.run(res))

sess.close()
##############################

In [ ]:
import numpy,scipy,time
import tensorflow as tf
import operateurs_t
#############################
def f(N):
    B = tf.constant(scipy.eye(N))
    with tf.device('/gpu:0'):
        #B_inv = tf.matrix_inverse(B)
        B_inv = tf.matmul(B,B)
        B_inv = tf.matmul(B_inv,B)
    return B_inv
##############################
N = 2000
##############################
sess = tf.Session()
res = f(N)
start = time.time()
print(len(sess.run(res)))
print("Exécution en secondes =", time.time()-start)
sess.close()
##############################

In [ ]:
import numpy,scipy,time
##############################
def f(N):
    b = scipy.eye(N)
    return numpy.dot(b,b)
##############################
N = 2000
##############################
start = time.time()
print(len(f(N)))
print("Exécution en secondes =", time.time()-start)

In [ ]:
import operateurs_t

def U(y,t):
    return 1-y**2

In [ ]:
N = 2001
[a,b] = operateurs_t.buildAB_forward(10000, 0.6, N, U, 1)

# numpy
start = time.time()
c = numpy.dot(a,b)
print("(Numpy) Exécution en secondes =", time.time()-start)


# tensorflow w/ placeholders
sess = tf.Session()

A = tf.placeholder(tf.complex128, shape=(N-4, N-4))
B = tf.placeholder(tf.complex128, shape=(N-4, N-4))

with tf.device('/gpu:0'):
    C = numpy.dot(A,B)
    #C = tf.matmul(A,B)
    
start = time.time()
c = sess.run(C, feed_dict={A:a,B:b})
print("(Tf placeh.) Exécution en secondes =", time.time()-start)

sess.close()


# tensorflow direct
#sess = tf.Session()
    
#start = time.time()
#c = tf.matmul(a,b*(1+0j))
#print("(Tf direct) Exécution en secondes =", time.time()-start)

#sess.close()